In [31]:
import pandas as pd

# Read in data, skipping description rows
rent = pd.read_excel("price_index_private_rent.xlsx", skiprows=7)

# Give the columns names that easier to work with
rent.columns = ["date", "la_code", "la", "region", "annual_change", "rent"]

# Set the date column as date time
rent["date"] = pd.to_datetime(rent["date"])

# Get the year
rent["year"] = rent["date"].dt.year

# Drop Wales, Scotland, Northern Ireland and United Kingdom
rent = rent[
    (~rent["region"].isin(["Wales", "Scotland", "Northern Ireland", "United Kingdom"]))
    & (rent["year"] >= 2020)
]

rent["rent"] = rent["rent"].astype(float)

In [32]:
deprivation = pd.read_excel("2025_deprivation_scores.xlsx", sheet_name="IoD2025 Scores")
deprivation.columns = [
    "lsoa_code",
    "lsoa_name",
    "la_code",
    "la_name",
    "imd",
    "income",
    "employment",
    "education",
    "health",
    "crime",
    "housing",
    "living_environment",
    "housing",
    "child",
    "older",
    "adult_skills",
    "geo_barriers",
    "wider",
    "indoors",
    "outdoors",
]
deprivation

,lsoa_code,lsoa_name,la_code,la_name,imd,income,employment,education,health,crime,housing,living_environment,housing,child,older,adult_skills,geo_barriers,wider,indoors,outdoors
0,E01000001,City of London 001A,E09000001,City of London,8.742,0.013,0.014,0.004,-1.771,-2.220,10.950,69.345,0.039,0.012,-2.902,0.030,4.437,0.688,1.207,1.414
1,E01000002,City of London 001B,E09000001,City of London,4.722,0.018,0.010,0.169,-1.549,-2.277,6.703,43.890,0.076,0.026,-1.830,0.032,3.589,-0.628,0.355,1.839
2,E01000003,City of London 001C,E09000001,City of London,9.250,0.107,0.064,3.269,-0.292,-0.765,9.735,41.163,0.250,0.153,-0.756,0.113,2.509,0.358,0.318,1.679
3,E01000005,City of London 001E,E09000001,City of London,19.884,0.211,0.104,17.852,0.436,-0.626,24.623,38.695,0.459,0.625,-0.814,0.305,2.914,4.091,0.012,2.065
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,25.307,0.343,0.120,25.442,-0.372,-0.072,38.025,29.616,0.640,0.156,-0.199,0.336,13.049,6.204,0.399,0.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33750,E01035758,Vale of White Horse 014H,E07000180,Vale of White Horse,7.945,0.101,0.074,10.893,-1.004,0.215,15.321,8.503,0.192,0.069,0.079,0.143,31.877,-0.124,-0.386,-0.309
33751,E01035759,Vale of White Horse 015G,E07000180,Vale of White Horse,7.622,0.097,0.081,7.528,-0.645,-1.047,23.669,5.731,0.162,0.091,-0.245,0.137,49.863,-1.960,-0.628,-0.313
33752,E01035760,Vale of White Horse 015H,E07000180,Vale of White Horse,7.946,0.170,0.067,10.064,-0.743,-0.157,10.074,3.450,0.206,0.316,0.084,0.122,31.757,-2.305,-1.794,-0.250
33753,E01035761,Vale of White Horse 015I,E07000180,Vale of White Horse,6.682,0.136,0.054,8.482,-0.938,-0.566,19.265,4.985,0.212,0.070,-0.054,0.116,44.211,-2.328,-1.459,-0.044


In [33]:
mismatches = [
    i for i in deprivation["la_code"].unique() if i not in rent["la_code"].unique()
]
print(mismatches)

['E09000001', 'E08000016', 'E08000019', 'E06000053']


In [34]:
remap = {
    "E08000016": "E08000038",  # Barnsley
    "E08000019": "E08000039",  # Sheffield
}
deprivation["la_code"] = deprivation["la_code"].replace(remap)

# Isles of Scilly and City of London
drop = ["E06000053", "E09000001"]
deprivation = deprivation[~deprivation["la_code"].isin(drop)]

In [35]:
df = rent.merge(
    deprivation.groupby(["la_code", "la_name"])[["imd", "income"]].mean().reset_index(),
    on="la_code",
    how="left",
)

inverse_map = {v: k for k, v in remap.items()}
df["la_code"] = df["la_code"].replace(inverse_map)

In [36]:
import plotly.express as px


def calculate_cumulative_pct_change(group):
    baseline = group.iloc[0]
    return ((group - baseline) / baseline) * 100


grouped_rent = df.groupby("la_code")["rent"]
df["cum_pct"] = grouped_rent.transform(calculate_cumulative_pct_change)

to_plot = df[["date", "la_name", "region", "cum_pct"]].copy()

regional_averages = to_plot.groupby(["region", "date"])["cum_pct"].mean().reset_index()
regional_averages["la_name"] = "Regional Average"

# Calculate national average
national_average = to_plot.groupby("date")["cum_pct"].mean().reset_index()
national_average["la_name"] = "National Average"

# Define region order
region_order = [
    "North West",
    "Yorkshire and The Humber",
    "North East",
    "West Midlands",
    "East Midlands",
    "East of England",
    "South West",
    "London",
    "South East",
]

# Set region as categorical with specified order
to_plot["region"] = pd.Categorical(
    to_plot["region"], categories=region_order, ordered=True
)
to_plot = to_plot.sort_values(["region", "date"])

to_plot = pd.concat([to_plot, regional_averages], ignore_index=True)

# Calculate endline percent change for each region and national average
endline_changes = (
    regional_averages.sort_values("date").groupby("region")["cum_pct"].last()
)
national_endline = national_average["cum_pct"].iloc[-1]

fig = px.line(
    to_plot,
    x="date",
    y="cum_pct",
    color="la_name",
    facet_col="region",
    facet_col_wrap=3,
    category_orders={"region": region_order},
    title="Private Rent Change Over Time by Local Authority",
    labels={"cum_pct": "", "date": "Date"},
    # width=1080,
    # height=768,
    # responsive=True,
)

# Set transparency for all lines but regional averages
# Corporate colors: rgb(10, 255, 206), rgb(244, 240, 231), rgb(21, 38, 58)
for trace in fig.data:
    if trace.name != "Regional Average":
        trace.update(
            line=dict(width=1),
            opacity=0.3,
            hovertemplate="<b>%{fullData.name}</b><br>%{y:.1f}%<extra></extra>",
        )
    else:
        trace.update(
            line=dict(color="rgb(21, 38, 58)", width=2),
            hovertemplate="<b>%{fullData.name}</b><br>%{y:.1f}%<extra></extra>",
        )

# Add hover behavior via layout
fig.update_layout(
    showlegend=False,
    hovermode="closest",
    hoverdistance=100,
    plot_bgcolor="rgb(244, 240, 231)",
    paper_bgcolor="rgb(244, 240, 231)",
    title=dict(
        text=f"<b>Cumulative rent change as a proportion of 2020 baseline</b><br>National average: {national_endline:.0f}%",
        y=0.965,
        yanchor="top",
    ),
    margin=dict(t=120),
)

# Lock axes and hide modebar
fig.update_xaxes(fixedrange=True)
fig.update_yaxes(fixedrange=True)

# Remove gridlines and update facet titles
fig.update_xaxes(
    showgrid=False, title_text="", tickfont=dict(size=12), title_standoff=25
)
fig.update_yaxes(
    showgrid=False,
    title_text="",
    range=[-20, 80],
    tickfont=dict(size=12),
    title_standoff=25,
)

# Change "region=X" to "X" in facet titles
fig.for_each_annotation(
    lambda a: a.update(
        text=f"{a.text.split('=')[-1]}<br>{endline_changes.get(a.text.split('=')[-1], 0):.0f}%",
        font=dict(size=14),
    )
)

fig.show()
fig.write_html("private_rent_change_by_la.html", config={"displayModeBar": False})

In [41]:
# Regional baseline/endline in absolute and relative terms
la_rent = df.groupby(["la_code", "date"])["rent"].mean().reset_index()

# Get baseline and endline for each region
la_baseline = la_rent.sort_values("date").groupby("la_code")["rent"].first()
la_endline = la_rent.sort_values("date").groupby("la_code")["rent"].last()

# Calculate changes
la_stats = pd.DataFrame(
    {
        "baseline": la_baseline,
        "endline": la_endline,
        "absolute": la_endline - la_endline,
        "pct_change": ((la_endline - la_baseline) / la_baseline) * 100,
    }
)

# Merge back to df
la_stats = (
    la_stats.sort_values("pct_change", ascending=False)
    .reset_index()
    .merge(df[["la_code", "la_name", "region", "imd", "income"]], how="left")
    .drop_duplicates()
    .assign(rent_risk_index=lambda x: x["pct_change"] * x["income"])
)

# Normalise index
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 100))
la_stats["rent_risk_index"] = scaler.fit_transform(la_stats[["rent_risk_index"]])

In [86]:
print(
    la_stats.sort_values("rent_risk_index", ascending=False)[
        ["la_name", "region", "rent_risk_index"]
    ]
    .assign(rent_risk_index=lambda x: x["rent_risk_index"].astype(int))
    .head(10)
    .to_markdown(index=False)
)

| la_name       | region        |   rent_risk_index |
|:--------------|:--------------|------------------:|
| Tameside      | North West    |               100 |
| Rochdale      | North West    |                99 |
| Oldham        | North West    |                94 |
| Manchester    | North West    |                93 |
| Liverpool     | North West    |                88 |
| Nottingham    | East Midlands |                87 |
| Birmingham    | West Midlands |                86 |
| Sandwell      | West Midlands |                86 |
| Salford       | North West    |                82 |
| Wolverhampton | West Midlands |                81 |


In [58]:
# Get regional ranks by deprivation (imd)
la_stats["imd_rank"] = la_stats["imd"].rank(method="first", ascending=False)

print(
    la_stats.sort_values("imd_rank", ascending=True)[
        ["la_name", "region", "imd_rank", "imd"]
    ].to_markdown(index=True)
)

|       | la_name                             | region                   |   imd_rank |      imd |
|------:|:------------------------------------|:-------------------------|-----------:|---------:|
| 18630 | Blackpool                           | North West               |          1 | 43.5275  |
| 11868 | Middlesbrough                       | North East               |          2 | 40.9287  |
|   759 | Manchester                          | North West               |          3 | 37.9629  |
| 20286 | Hartlepool                          | North East               |          4 | 37.8621  |
|  3795 | Birmingham                          | West Midlands            |          5 | 37.4801  |
| 13593 | Hastings                            | South East               |          6 | 37.4229  |
| 12213 | Kingston upon Hull, City of         | Yorkshire and The Humber |          7 | 37.0708  |
|   621 | Liverpool                           | North West               |          8 | 36.963   |
| 13455 | 

In [ ]:
la_stats["imd_rank"].max()

In [59]:
print(
    la_stats.sort_values("pct_change", ascending=False)[
        ["la_name", "region", "pct_change", "imd_rank"]
    ]
    .head(10)
    .assign(pct_change=lambda x: x["pct_change"].astype(int).astype(str) + "%")
    .to_markdown(index=False)
)

| la_name              | region     | pct_change   |   imd_rank |
|:---------------------|:-----------|:-------------|-----------:|
| Tameside             | North West | 59%          |         45 |
| Rossendale           | North West | 53%          |         59 |
| Rochdale             | North West | 51%          |         20 |
| Bury                 | North West | 51%          |        104 |
| Trafford             | North West | 48%          |        201 |
| Salford              | North West | 48%          |         27 |
| Oldham               | North West | 47%          |         14 |
| Folkestone and Hythe | South East | 47%          |         73 |
| Stockport            | North West | 45%          |        164 |
| Liverpool            | North West | 45%          |          8 |


In [8]:
import geopandas as gpd
import pyproj

# Load shapefile - update the path to your shapefile
gdf = gpd.read_file(
    "Local_Authority_Districts_May_2024_Boundaries_UK_BFE_-5758551109064458912/LAD_MAY_2024_UK_BFE.shp"
)

# Merge with your data (assuming area_code matches a column in the shapefile)
gdf = gdf.merge(la_stats, left_on="LAD24CD", right_on="la_code", how="right")

gdf.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)

# Simplify geometries for better performance
gdf["geometry"] = gdf.to_crs(gdf.estimate_utm_crs()).simplify(750).to_crs(gdf.crs)

In [ ]:
import plotly.express as px


# Create custom color scale using corporate colors
# rgb(21, 38, 58) for low, rgb(10, 255, 206) for high
colorscale = [
    [0, "rgb(244, 240, 231)"],  # Beige for lowest
    [0.5, "rgb(10, 255, 206)"],  # Cyan for highest
    [1, "rgb(21, 38, 58)"],  # Dark blue for medium
]

subtitle = "% change in private rent (2020-2025) weighted by average income deprivation score (2025).<br>Higher values indicate greater risk of rent affordability issues."

fig = px.choropleth(
    gdf,
    geojson=gdf.geometry.__geo_interface__,
    locations=gdf.index,
    color="rent_risk_index",
    color_continuous_scale=colorscale,
    labels={"rent_risk_index": "Rent Risk Index"},
    title=f"<b>Rent Risk Index by Local Authority</b><br><sub>{subtitle}</sub>",
    hover_data={
        "la_name": True,
        "region": True,
        "pct_change": ":.1f",
        "income": ":.2f",
        "rent_risk_index": ":.1f",
    },
)

fig.update_traces(
    hovertemplate="<b>%{customdata[0]}</b><br>Region: %{customdata[1]}<br>Rent Change: %{customdata[2]:.1f}%<br>Income Deprivation: %{customdata[3]:.2f}<br>Risk Index: %{customdata[4]:.1f}<extra></extra>"
)

fig.update_geos(
    fitbounds="locations",
    visible=False,
    projection_type="mercator",
    bgcolor="rgb(244, 240, 231)",
)

fig.update_layout(
    # height=800,
    # width=1000,
    margin={"r": 100, "t": 150, "l": 0, "b": 100},
    paper_bgcolor="rgb(244, 240, 231)",
    plot_bgcolor="rgb(244, 240, 231)",
    title=dict(y=0.95, yanchor="top"),
)

fig.show(config={"displayModeBar": False})
fig.write_html("rent_risk_index_map.html", config={"displayModeBar": False})

In [ ]:
# Scatter plot with overall trendline: IMD vs Rent Change
fig = px.scatter(
    la_stats,
    x="imd",
    y="pct_change",
    color="region",
    hover_data=["la_name"],
    title="<b>Higher deprivation scores are associated with faster rent increases</b>",
    labels={
        "imd": "Index of Multiple Deprivation (IMD)",
        "pct_change": "Rent Change 2020-2025 (%)",
    },
    trendline="ols",
    trendline_scope="overall",
)

# Get trendline results
results = px.get_trendline_results(fig)
slope = results.px_fit_results.iloc[0].params[1]
print(f"Slope: {slope:.3f}")
print(
    f"For every 1 unit increase in deprivation (IMD), rent increases by approximately {slope:.1f} percentage points on average."
)

# Update styling to match corporate colors
fig.update_traces(
    marker=dict(size=8, opacity=0.7),
    hovertemplate="<b>%{customdata[0]}</b><br>Region: %{fullData.name}<br>IMD: %{x:.2f}<br>Rent Change: %{y:.1f}%<extra></extra>",
    selector=dict(mode="markers"),
)

# Update trendline styling
fig.update_traces(
    line=dict(color="black", dash="dot", width=2),
    opacity=0.5,
    hovertemplate="Trendline<extra></extra>",
    selector=dict(mode="lines"),
)

# Style layout
fig.update_layout(
    plot_bgcolor="rgb(244, 240, 231)",
    paper_bgcolor="rgb(244, 240, 231)",
    font=dict(size=12),
    title=dict(y=0.95, yanchor="top"),
    margin=dict(t=80, l=60, r=60, b=120),
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2,
        xanchor="center",
        x=0.5,
        title="",
        itemsizing="constant",
        tracegroupgap=0,
    ),
)

# Remove gridlines
fig.update_xaxes(showgrid=False, title_standoff=15)
fig.update_yaxes(showgrid=False, title_standoff=15)

fig.show()
fig.write_html("imd_rent_scatter.html", config={"displayModeBar": False})

Slope: 0.204
For every 1 unit increase in deprivation (IMD), rent increases by approximately 0.2 percentage points on average.
